In [ ]:
import cv2
import numpy as np
import time
from tkinter import *
from IPython.display import clear_output

class marker:
    
    def __init__(self, cam_num, cellx, celly, colors):
        #self.img = img
        self.cellx = cellx
        self.celly = celly
        self.colors = colors
        
        cam = cv2.VideoCapture(cam_num)
        flag, img = cam.read()
        
        shape = img.shape
        
        width = shape[1]
        height = shape[0]
        
#         print('width = ', width)
#         print('height = ', height)
        
        self.width = width
        self.height = height
        
        size_cell_x = width//cellx
        size_cell_y = height//celly
        
        self.size_cell_x = size_cell_x
        self.size_cell_y = size_cell_y
        
#         print('size_cell_x = ', size_cell_x)
#         print('size_cell_y = ', size_cell_y)
        
    def draw_grid(self, img):
        i = 0
        j = 0   
        while i < self.width:
            while j < self.height:
                cv2.rectangle(img,(i, j),(i+self.size_cell_x, j+self.size_cell_y),(255,255,255),1)
                j+= self.size_cell_y
            i+= self.size_cell_x
            j = 0
    
    def detect_marker (self, img, color):
        #color = [hsv_min, hsv_max]

        # преобразуем RGB картинку в HSV модель
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # применяем цветовой фильтр
        thresh = cv2.inRange(hsv, color[0], color[1])

        # вычисляем моменты изображения
        moments = cv2.moments(thresh, 1)
        dM01 = moments['m01']
        dM10 = moments['m10']
        dArea = moments['m00']
        # будем реагировать только на те моменты,
        # которые содержать больше 100 пикселей

        marker_coord = [0, 0]
        if dArea > 100:
            x = int(dM10 / dArea)
            y = int(dM01 / dArea)
            marker_coord[0] = x
            marker_coord[1] = y
            #cv2.circle(img, (x, y), 10, (0,0,255), -1)
            
        return marker_coord
    
    
    def find_marker_place (self, img, marker_coord):
        coord_rectang = [0, 0, 0, 0]
        k_i = 0
        l_j = 0
        while k_i <= self.width:
            while l_j <= self.height:
                if marker_coord[0] < k_i + self.size_cell_x and marker_coord[0] > k_i and marker_coord[1] < l_j + self.size_cell_y and marker_coord[1] > l_j:
                    coord_rectang[0] = k_i
                    coord_rectang[1] = l_j
                    coord_rectang[2] = k_i+self.size_cell_x
                    coord_rectang[3] = l_j+self.size_cell_y
                    #cv2.rectangle(img,(k_i, l_j),(k_i+self.size_cell_x, l_j+self.size_cell_y),(0,255,255),3)                  
                l_j+= self.size_cell_y
            k_i+= self.size_cell_x
            l_j = 0

        return coord_rectang   

In [ ]:
#объявление цветов
hsv_min = np.array((53, 55, 147), np.uint8)
hsv_max = np.array((83, 160, 255), np.uint8)

hsv_min1 = np.array((108, 115, 192), np.uint8)
hsv_max1 = np.array((79, 69, 147), np.uint8)

#вручную засунуть их сюда
colors = [[hsv_min, hsv_max], [hsv_min1, hsv_max1]]

#выбор на сколько клеток разбить экран
cellx = 8
celly = 8 

#номер камеры
cam_num = 0
cam = cv2.VideoCapture(cam_num)

mark = marker(cam_num, cellx, celly, colors)

cv2.namedWindow ("result", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("result", (mark.width, mark.height))


while True:
    flag, img = cam.read()
    mark.draw_grid(img)
    
    marker_coord = mark.detect_marker(img, colors[0])
    
    marker_coord_2 = mark.detect_marker(img, colors[1])

    if marker_coord != [0, 0]:
        cv2.circle(img, (marker_coord[0], marker_coord[1]), 10, (0,0,255), -1)
    
    if marker_coord_2 != [0, 0]:
        cv2.circle(img, (marker_coord_2[0], marker_coord_2[1]), 10, (0,0,255), -1) 
    
    coord_rectang = mark.find_marker_place(img, marker_coord)
    coord_rectang_2 = mark.find_marker_place(img, marker_coord_2)

    cv2.rectangle(img,(coord_rectang[0], coord_rectang[1]),(coord_rectang[2], coord_rectang[3]),(0,255,255),3)   
    cv2.rectangle(img,(coord_rectang_2[0], coord_rectang_2[1]),(coord_rectang_2[2], coord_rectang_2[3]),(0,0,255),3)   
    
    
    keyb = cv2.waitKey(1)
    cv2.imshow('result', img)
    
    if (keyb & 0xFF == ord ('q')):
           break

cam.release()
cv2.destroyAllWindows()